In [1]:
import logging
import json
import os
from urllib.request import urlopen
import json
import numpy as np
from scipy.spatial import distance
import math
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
import pandas as pd
import logging
import shutil
from datetime import datetime
import warnings
import time
import pika

#import azure.functions as func
from azure.data.tables import TableClient
from azure.data.tables import UpdateMode
from azure.storage.filedatalake import DataLakeServiceClient
from azure.storage.filedatalake import FileSystemClient
from azure.core.exceptions import HttpResponseError

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [3]:
WRITE_CONNECTION_STRING = "DefaultEndpointsProtocol=https;AccountName=nbadatalakedev;AccountKey=Gs49jwtBv2AaK6MTJnlc2iiqc1yCbZKVliwGveYkjbF+f1mjSukpnUk07RaWkCMqP4VOk5bh+bucVtMdhT3x2g==;EndpointSuffix=core.windows.net"
CONFIG_CONNECTION_STRING = "DefaultEndpointsProtocol=https;AccountName=quintardatalakedev;AccountKey=J5YxEVt+SWVmr9LOtoImHSTRgQvciUbjBo063PTdUsJvmwjBEnYvpNwg6WklPXqW1d+fO0dQRpHq+AStNTBFew==;EndpointSuffix=core.windows.net"
FILE_SYSTEM = "mba"
JSON_FILE_PATH = "sdk/phobos_slam"
FILE_NAME = "qrealityBasketball_test3.json"


In [4]:
def getParams():
    dataLakeClient = DataLakeServiceClient.from_connection_string(conn_str=CONFIG_CONNECTION_STRING)
    fileSystemClient = dataLakeClient.get_file_system_client(file_system=FILE_SYSTEM)
    directoryClient = fileSystemClient.get_directory_client(JSON_FILE_PATH)
    configFile = directoryClient.get_file_client(FILE_NAME)

    download = configFile.download_file()
    dataBytes = download.readall()
    dataStr = dataBytes.decode("utf-8")
    jsonData = json.loads(dataStr)
    shotTrailData = jsonData['sportData']
    return shotTrailData

In [5]:
class liveInfo:
    def __init__(self, gameID, quarter):
        self.gameID = gameID
        urlInfo = str(str(self.gameID) + "_" + str(quarter))
        self.url = "http://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2021/scores/pbp/" + urlInfo + "_pbp.json"
        #self.url = "http://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2021/scores/pbp/0022101103_1_pbp.json"
        logging.info(self.url)
        while True:
            try:
                c = urlopen(self.url)
                break
            except:
                logging.info("WAITING FOR QUARTER TO START")
                time.sleep(120)
                #continue
        

    def getData(self, evtID, de):
        while(True):
            response = urlopen(self.url)
            spbpData = json.loads(response.read())["g"]
            period = spbpData["p"]
            self.nextURL = spbpData["next"]
            #newData = [evt for evt in spbpData["pla"] if int(evt["evt"]) > evtID]
            if "pla" in spbpData:
                newData = [evt for evt in spbpData["pla"] if int(evt["evt"]) > evtID]
            else:
                newData = []
            if newData != []:
                break
            else:
                logging.info("WAITING FOR NEW DATA")
            time.sleep(1)
        #newData = spbpData["pla"]
        numEvts = len(newData)
        #logging.info("NEW DATA")
        #logging.info(newData)
        self.events = []
        self.tids = []
        if numEvts > 0:
            for event in newData:
                if len(self.tids) == 0:
                    if event["tid"] != 0:
                        if (event["tid"]) != int(event["oftid"]):
                            self.tids.append(event["tid"])
                            self.tids.append(event["oftid"])
                evtID = event["evt"]
                de = event["de"].lower()
                relData = {"gid": self.gameID, "eid": evtID, "pe": period, "tid": event["tid"], "pid": event["pid"], "epid": event["epid"], "opid": event["opid"] , "tr": event["cl"], "x": event["locX"], "y": event["locY"], "de": de}
                #logging.info("RELDATAT")
                #logging.info(relData)
                self.events.append(relData)
                if de == "end period":
                    break
            #logging.info(self.events)
        return evtID, period, de.lower()


class shotInfo: ## Get SHOTS, SCORING
    def __init__(self, game_ID, hTeam):
        self.game_ID = game_ID
        self.hTeam = hTeam

    def func(self, x, a, b, c):
        return a*x**2+b*x+c

    def heights(self, maxH):
        diff = 0
        max_point = round(len(self.z_pts) * 0.5)
        self.z_pts[-1] = 100.0
                
        maxVal = maxH
        maxDiff = maxVal - 100.0
        self.z_pts[max_point] = maxVal
        diff = maxVal - self.z_pts[0]
        for i in range(1, max_point):
            self.z_pts[i] = round((self.z_pts[i-1] + (diff/max_point)), 2)
        for i in range(max_point+1, len(self.z_pts)):
            self.z_pts[i] = round((self.z_pts[i-1] - ((maxDiff) / ((len(self.z_pts) - max_point) - 1))), 2)

        x = list(range(0, len(self.z_pts)))
        params, _ = curve_fit(self.func, x, self.z_pts)
        a, b, c = params[0], params[1], params[2]
        for i in x:
            self.z_pts[i] = round(((a*i**2)+(b*i)+c), 2)

        hDiff = 100.0 - self.z_pts[-1]
        for i in range(0, len(self.z_pts)):
            self.z_pts[i] += hDiff
            self.z_pts[i] = round(self.z_pts[i], 2)

    
    def getTrace(self, pid):
        pc_url = "http://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2021/players/playercard_" + str(pid) + "_02.json"
        response = urlopen(pc_url)
        playerData = json.loads(response.read())
        params = getParams()
        shotTrailData = params['shotTrails']
        offset = shotTrailData['startHeightOffset']
        height = (int(playerData["pl"]["ht"].split("-")[0]) * 10) + math.floor((float(playerData["pl"]["ht"].split("-")[1])/12)* 10) + offset

        if self.st == "ft":
            p1 = (0, self.pen, height)
        else:
            p1 = (self.x, self.y, height)
        p2 = (self.p[0], self.p[1], 100)
        dist_points = (p1[0], p1[1])
        origin = (self.p[0], self.p[1])

        num_points = shotTrailData['numPoints']

        self.x_pts = []
        self.y_pts = []
        self.z_pts = []

        t = 0.0;
        delta_t = float(1.0)/ float(num_points)

        for ipoint in range(0, num_points+1):
            x = self.p[0]
            y = self.p[1] - 430.0
            z = 100
            xi = round((x + (t * float(p1[0]))), 2)
            yi = round((y + (t * float(p1[1]))), 2)
            zi = 0.0
            self.x_pts.append(xi)
            self.y_pts.append(yi)
            self.z_pts.append(zi)
            t += delta_t
        self.z_pts[-1] = float(p1[2])
        self.x_pts.reverse()
        self.y_pts.reverse()
        self.z_pts.reverse()
        if self.st == "ft":
            maxH = shotTrailData['apex_ft']
        elif self.st == "fg":
            maxH = shotTrailData['apex_2pt']
        elif self.st == "3pt":
            maxH = shotTrailData['apex_3pt']
        self.heights(maxH)
        ynp = np.array(self.y_pts)
        xnp = np.array(self.x_pts)
        if self.tName == self.hTeam.lower(): #'mia':
            ynp = np.multiply(ynp, -1.0)
            xnp = np.multiply(xnp, -1.0)
        self.x_pts = xnp.tolist()
        self.y_pts = ynp.tolist()
        combList = self.y_pts + self.x_pts + self.z_pts
        combList = np.divide(combList, 10.0)
        combList = np.around(combList, 3)
        self.trace = combList.tolist()
        self.xy_pts = [self.y_pts[0], self.x_pts[0]]


    def getEvtData(self, tids, qData, shots, points, assists, blocks, rebounds, steals):
        shots = []
        self.qData = qData
        for evt in self.qData:
            if evt["tid"] == tids[0]:
                tid = tids[1]
            else:
                tid = tids[0]
            if ("shot:" in evt["de"]) or ("free throw" in evt["de"]):
                if "missed" in evt["de"]:
                    ma = 0
                else:
                    ma = 1
                if "3pt" in evt["de"]:
                    self.st = "3pt"
                elif "free throw" in evt["de"]:
                    self.st = "ft"
                else:
                    self.st = "fg"

                self.x = float(evt['x'])
                self.y = float(evt['y'])
                self.tName = evt["de"].split('[')[1][0:3]
                self.p = [0.0, 12.5]
                self.pen = 150.0

                self.getTrace(evt["pid"])
                event = { "gid": self.game_ID, "eid": evt["eid"], "pe": evt["pe"], "tid": evt["tid"], "pid": evt["pid"], "tr": evt["tr"], "ma": ma, "st": self.st, "x": self.xy_pts[0], "y": self.xy_pts[1], "trace": self.trace}
                if ma == 1:
                    point = int((evt["de"].split("(")[1]).split(" ")[0])
                    points[evt["tid"]][evt["pid"]] = point
                    if "assist" in evt["de"]:
                        assist = int((((evt["de"].split("assist:")[1]).split("("))[1]).replace(" ast)", ""))
                        assists[evt["tid"]][int(evt["epid"])] = assist
                shots.append(event)
            if "rebound" in evt["de"]:
                logging.info(evt["de"])
                logging.info(evt["pid"])
                if evt["pid"] in rebounds[evt["tid"]]:
                    rebounds[evt["tid"]][evt["pid"]] += 1
                else:
                    rebounds[evt["tid"]][evt["pid"]] = 1
                if "phi" in evt["de"]:
                    logging.info(rebounds)
            if "block" in evt["de"]:
                block = int((evt["de"].split("(")[1]).split(" ")[0])
                blocks[tid][evt["opid"]] = block
            if "steal" in evt["de"]:
                steal = int((evt["de"].split("(")[1]).split(" ")[0])
                steals[tid][evt["opid"]] = steal
        return shots, points, assists, blocks, rebounds, steals


    def leaderBoard(self, tids, points, assists, blocks, rebounds, steals):
        leaderboardData = []
        cats = ["PTS", "AST", "BLK", "REB", "STL"]
        for tid in tids:
            teamLeaders = []
            points[tid].pop(0, None)
            assists[tid].pop(0, None)
            blocks[tid].pop(0, None)
            rebounds[tid].pop(0, None)
            steals[tid].pop(0, None)
            inArr = [points[tid], assists[tid], blocks[tid], rebounds[tid], steals[tid]]
            for i in range(0, 5):
                if (len(inArr[i]) >= 1) and (int(max(inArr[i], key=inArr[i].get)) != 0):
                    pid = max(inArr[i], key=inArr[i].get)
                    scr = int(inArr[i][max(inArr[i], key=inArr[i].get)])
                    pidStr = str(pid)
                    pc_url = "http://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2021/players/playercard_" + pidStr + "_02.json"
                    #logging.info(pc_url)
                    #pc_url = "http://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2021/players/playercard_1628378_02.json"
                    response = urlopen(pc_url)
                    playerData = json.loads(response.read())
                    #playerData = {"pl": {"ln": "Parsons", "fn": "Jim"}}
                    pl_hs = "https://ak-static.cms.nba.com/wp-content/uploads/headshots/nba/latest/260x190/" + str(pid) + ".png"
                    leader = {"cat": cats[i], "fn": playerData["pl"]["fn"], "sn": playerData["pl"]["ln"] ,"pid": int(pid), "scr": scr, "hs": pl_hs}
                else:
                    pid = 0
                    scr = 0
                    leader = {"cat": cats[i], "fn": "N/A", "sn": "N/A" ,"pid": pid, "scr": scr, "hs": ""}
                teamLeaders.append(leader)
            leaderboardData.append({"tid": tid, "teamLeaders": teamLeaders})
        return leaderboardData


In [6]:
def writeTable(gameData, connection_string, gameID):
    shots = gameData["shots"]
    leaders = gameData["leaderboard"]
    #logging.info(gameData)
    operations = []
    for shot in shots:
        #logging.info(shot)
        shot['PartitionKey'] = str(gameID)
        shot['RowKey'] = str(shot['eid'])
        shot['trace'] = "".join(str(shot['trace']))
        operations.append(("upsert", shot))

    with TableClient.from_connection_string(connection_string, table_name="GameChronicle") as table:
        try:
            logging.info("WRITING TO TABLE")
            try:
                for i in range(0, len(operations), 100):
                    operation = list(operations[i:i+100])
                    lastTask = operation[-1]
                    #logging.info(f'LAST TASK = {lastTask}')
                    #logging.info(operations[99])
                    table.submit_transaction(operation)
                logging.info("SUBMIT TRANSACTION PASSED")
            except Exception as e:
                logging.info("SUBMIT TRANSACTION FAILED")
                lenOp = len(operations)
                logging.info(f"LENGTH OF OPERATIONS = {lenOp}")
                logging.info(f"EXCEPTION : {e}")
            
        except:
            updateStatus = 0
            return ("Failed to add/update game chronicles")            
            
    for team in leaders:
        if team:
            for leader in team['teamLeaders']:
                logging.info(leader)
                leader['PartitionKey'] = str(gameID)
                leader['RowKey'] = str(team['tid']) + '_' + leader['cat'] # + '_' + str(eid)
                if leader:
                    with TableClient.from_connection_string(connection_string, table_name="LeaderBoard") as table:
                        try:
                            #add new entity if it does not exist. update if the entity exists
                            createdEntity = table.upsert_entity(mode=UpdateMode.MERGE, entity=leader)
                            #logging.info(createdEntity)
                            #Return http response
                            updateStatus = 1
                        except HttpResponseError:
                            updateStatus = 0
                            return ("Failed to add/update game leaders")
                else:
                    return ("Failed to add/update game leaders")
    return len(shots)

In [7]:
def pub2Queue(jsonData, gameID):
    credentials = pika.PlainCredentials('quintar', 'quintar123')
    #parameters = pika.ConnectionParameters('20.231.253.196', 5672, '/', credentials)
    parameters = pika.ConnectionParameters('20.232.3.112', 5672, '/', credentials)
    connection = pika.BlockingConnection(parameters)
    channel = connection.channel()

    channel.exchange_declare(exchange='tq6_exchange', exchange_type='fanout')
    channel.queue_declare(queue='testq6', arguments={'x-message-ttl' : 30000})
    
    message = json.dumps(jsonData)

    channel.basic_publish(exchange='tq6_exchange', routing_key=str(gameID), body=message)
    logging.info(" [x] Sent %r" % message)
    connection.close()

In [8]:
def main():
    logging.info('game chronicles received a request')
    connection_string = WRITE_CONNECTION_STRING
    league = 'nba'
    params = getParams()
    logging.info(params)
    gameID = params['gid']
    for i in range(0, 3):
        logging.info(f"GAME ID = {gameID}")
    evtID = 0
    #gameID = '0042100235'
    
    try:
        logging.info("HERE I AM!")
        logging.info("HERE I AM!")
        logging.info("HERE I AM!")
        logging.info("HERE I AM!")
        logging.info("THERE YOU GO!")
        gameID_orig = gameID
        with TableClient.from_connection_string(connection_string, table_name="Game") as table:
            logging.info("STEP-1")
            gamesFilter = f"PartitionKey eq '{gameID}'"
            gameEntity = table.query_entities(query_filter=gamesFilter)
            logging.info("STEP-2")
            count = 0
            logging.info(f'COUNT = {count}')
            logging.info((gameEntity))
            for entity in gameEntity:
                count += 1
                logging.info(f'COUNT = {count}')
                entity.pop("RowKey", None)
                entity.pop("PartitionKey", None)
                gameDetails = entity
                logging.info("STEP-3")
        tids = [gameDetails["hid"], gameDetails["vid"]]
        hTid = gameDetails["hid"]
        logging.info("WE LAND HERE???? ARE YOU SURE??")
    except:
        logging.info("ARE WE IN TROUBLE????")
        return (f"Failed to get games data")
    
    logging.info("HERE I AM!")
    gameID = gameID_orig
    
    try:
        with TableClient.from_connection_string(connection_string, table_name="Team") as table:
            teams = []
            teamFilter = f"PartitionKey eq 'nba' and RowKey eq '{hTid}'"
            teamEntities = table.query_entities(query_filter=teamFilter)
            for teamEntity in teamEntities:
                hTeam = teamEntity["ab"]
    except:
        return (f"Failed to get home team data")
    
    try:
        evtCount = 0
        quarter = 1
        de = "start period"
        gameOn = True
        shots = []
        for i in range(0, 10):
            logging.info(de)
            
        points = {tids[0]: {}, tids[1]: {}}
        assists = {tids[0]: {}, tids[1]: {}}
        blocks = {tids[0]: {}, tids[1]: {}}
        rebounds = {tids[0]: {}, tids[1]: {}}
        steals = {tids[0]: {}, tids[1]: {}}
        shotInfoObj = shotInfo(gameID, hTeam)
        quarter = 1
        evtID = 0
        period = 1
        de = "start period"
        numEvts = 0
        logging.info(de)
        for i in range(0, 2160):
            try:
                gameStatus = "GAMEON = " + str(gameOn)
                liObj = liveInfo(gameID, quarter)
                evtID, period, de = liObj.getData(evtID, de)
                nxtUrl = liObj.nextURL
                #logging.info("LIVE INFO")
                #logging.info(liObj.events)
                shots, points, assists, blocks, rebounds, steals = shotInfoObj.getEvtData(tids, liObj.events,  shots, points, assists, blocks, rebounds, steals)
                leaders = shotInfoObj.leaderBoard(tids, points, assists, blocks, rebounds, steals)
                printStr = "DESCRIPTION : " + de.lower()
                evtIDstr = "EVENT ID = " + str(evtID)
                logging.info(printStr)
                logging.info(evtIDstr)    
                json_data = {"shots": shots, "leaderboard": leaders}
                lenShots = len(shots)
                logging.info(f"LENGTH OF SHOTS = {lenShots}")
                if len(shots) > 0:
                    logging.info("WRITING NOW!!!!!!")
                    #pub2Queue(json_data, gameID)
                    evtCount += writeTable(json_data, connection_string, gameID)
                logging.info(gameStatus)
                if (de.lower() == "end period" and period != 4):
                    logging.info("CHANGE PERIOD")
                    quarter += 1
                lastEvt = "LAST EVENT = " + str(evtID)
                logging.info(lastEvt)
                if (period == 4 and de.lower() == "end period"):
                    gameOn = False
                    break
                time.sleep(1)
            except HttpResponseError as e:
                time.sleep(5)
            
            
        logging.info(f"HOME TEAM = {hTeam}")
        return (f'Game Chronicles Data uploaded. No. of Events = {evtCount}')
    except HttpResponseError as e:
        logging.error(e)
        return ("Failed to add game-chronicles information")

In [14]:
if __name__ == "__main__":
    main()

INFO:root:game chronicles received a request
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://quintardatalakedev.blob.core.windows.net/mba/sdk/phobos_slam/qrealityBasketball_test3.json'
Request method: 'GET'
Request headers:
    'x-ms-range': 'REDACTED'
    'x-ms-version': 'REDACTED'
    'Accept': 'application/xml'
    'User-Agent': 'azsdk-python-storage-dfs/12.5.0 Python/3.9.0 (Linux-5.4.0-77-generic-x86_64-with-glibc2.31)'
    'x-ms-date': 'REDACTED'
    'x-ms-client-request-id': '0437ffc4-dd61-11ec-b1be-ac1f6bac234a'
    'Authorization': 'REDACTED'
No body was attached to the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 206
Response headers:
    'Content-Length': '8530'
    'Content-Type': 'application/json'
    'Content-Range': 'REDACTED'
    'Last-Modified': 'Fri, 27 May 2022 00:58:25 GMT'
    'Accept-Ranges': 'REDACTED'
    'ETag': '"0x8DA3F7C0155E9F7"'
    'Server': 'Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0'
    'x

INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 206
Response headers:
    'Content-Length': '8530'
    'Content-Type': 'application/json'
    'Content-Range': 'REDACTED'
    'Last-Modified': 'Fri, 27 May 2022 00:58:25 GMT'
    'Accept-Ranges': 'REDACTED'
    'ETag': '"0x8DA3F7C0155E9F7"'
    'Server': 'Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0'
    'x-ms-request-id': '2afa1a17-201e-00a4-336d-7142a1000000'
    'x-ms-client-request-id': '06358188-dd61-11ec-8fbe-ac1f6bac234a'
    'x-ms-version': 'REDACTED'
    'x-ms-creation-time': 'REDACTED'
    'x-ms-blob-content-md5': 'REDACTED'
    'x-ms-lease-status': 'REDACTED'
    'x-ms-lease-state': 'REDACTED'
    'x-ms-blob-type': 'REDACTED'
    'x-ms-server-encrypted': 'REDACTED'
    'Date': 'Fri, 27 May 2022 02:02:12 GMT'
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://quintardatalakedev.blob.core.windows.net/mba/sdk/phobos_slam/qrealityBasketball_test3.json'
Request method: 'GET'
Request he

INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 206
Response headers:
    'Content-Length': '8530'
    'Content-Type': 'application/json'
    'Content-Range': 'REDACTED'
    'Last-Modified': 'Fri, 27 May 2022 00:58:25 GMT'
    'Accept-Ranges': 'REDACTED'
    'ETag': '"0x8DA3F7C0155E9F7"'
    'Server': 'Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0'
    'x-ms-request-id': 'eb6aafe2-b01e-0099-2a6d-7134ba000000'
    'x-ms-client-request-id': '086fa2cf-dd61-11ec-a1e2-ac1f6bac234a'
    'x-ms-version': 'REDACTED'
    'x-ms-creation-time': 'REDACTED'
    'x-ms-blob-content-md5': 'REDACTED'
    'x-ms-lease-status': 'REDACTED'
    'x-ms-lease-state': 'REDACTED'
    'x-ms-blob-type': 'REDACTED'
    'x-ms-server-encrypted': 'REDACTED'
    'Date': 'Fri, 27 May 2022 02:02:16 GMT'
INFO:root:[gsw] curry rebound (off:0 def:1)
INFO:root:201939
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://quintardatalakedev.blob.core.windows.net/mba/sdk/phobos_slam/qr

INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 206
Response headers:
    'Content-Length': '8530'
    'Content-Type': 'application/json'
    'Content-Range': 'REDACTED'
    'Last-Modified': 'Fri, 27 May 2022 00:58:25 GMT'
    'Accept-Ranges': 'REDACTED'
    'ETag': '"0x8DA3F7C0155E9F7"'
    'Server': 'Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0'
    'x-ms-request-id': 'd6c9dcc3-e01e-0094-326d-71fc6e000000'
    'x-ms-client-request-id': '0aa61f37-dd61-11ec-9ee3-ac1f6bac234a'
    'x-ms-version': 'REDACTED'
    'x-ms-creation-time': 'REDACTED'
    'x-ms-blob-content-md5': 'REDACTED'
    'x-ms-lease-status': 'REDACTED'
    'x-ms-lease-state': 'REDACTED'
    'x-ms-blob-type': 'REDACTED'
    'x-ms-server-encrypted': 'REDACTED'
    'Date': 'Fri, 27 May 2022 02:02:20 GMT'
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://quintardatalakedev.blob.core.windows.net/mba/sdk/phobos_slam/qrealityBasketball_test3.json'
Request method: 'GET'
Request he

INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 206
Response headers:
    'Content-Length': '8530'
    'Content-Type': 'application/json'
    'Content-Range': 'REDACTED'
    'Last-Modified': 'Fri, 27 May 2022 00:58:25 GMT'
    'Accept-Ranges': 'REDACTED'
    'ETag': '"0x8DA3F7C0155E9F7"'
    'Server': 'Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0'
    'x-ms-request-id': 'f115774f-001e-0025-2b6d-71e27b000000'
    'x-ms-client-request-id': '0d0a1568-dd61-11ec-af64-ac1f6bac234a'
    'x-ms-version': 'REDACTED'
    'x-ms-creation-time': 'REDACTED'
    'x-ms-blob-content-md5': 'REDACTED'
    'x-ms-lease-status': 'REDACTED'
    'x-ms-lease-state': 'REDACTED'
    'x-ms-blob-type': 'REDACTED'
    'x-ms-server-encrypted': 'REDACTED'
    'Date': 'Fri, 27 May 2022 02:02:24 GMT'
INFO:root:[gsw] wiggins rebound (off:0 def:1)
INFO:root:203952
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://quintardatalakedev.blob.core.windows.net/mba/sdk/phobos_slam/

...

INFO:root:WAITING FOR NEW DATA
INFO:root:WAITING FOR NEW DATA
INFO:root:WAITING FOR NEW DATA
INFO:root:WAITING FOR NEW DATA
INFO:root:WAITING FOR NEW DATA
INFO:root:WAITING FOR NEW DATA
INFO:root:WAITING FOR NEW DATA
INFO:root:WAITING FOR NEW DATA
INFO:root:WAITING FOR NEW DATA
INFO:root:WAITING FOR NEW DATA
INFO:root:WAITING FOR NEW DATA
INFO:root:WAITING FOR NEW DATA
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://quintardatalakedev.blob.core.windows.net/mba/sdk/phobos_slam/qrealityBasketball_test3.json'
Request method: 'GET'
Request headers:
    'x-ms-range': 'REDACTED'
    'x-ms-version': 'REDACTED'
    'Accept': 'application/xml'
    'User-Agent': 'azsdk-python-storage-dfs/12.5.0 Python/3.9.0 (Linux-5.4.0-77-generic-x86_64-with-glibc2.31)'
    'x-ms-date': 'REDACTED'
    'x-ms-client-request-id': 'cf67fe91-dd6b-11ec-9a7f-ac1f6bac234a'
    'Authorization': 'REDACTED'
No body was attached to the request
INFO:azure.core.pipeline.policies.http_logging_polic

INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 204
Response headers:
    'Cache-Control': 'no-cache'
    'Content-Length': '0'
    'ETag': 'W/"datetime'2022-05-27T03%3A19%3A29.1963749Z'"'
    'Server': 'Windows-Azure-Table/1.0 Microsoft-HTTPAPI/2.0'
    'x-ms-request-id': '4b080be1-4002-004d-7578-718de1000000'
    'x-ms-client-request-id': 'd18fcfce-dd6b-11ec-8741-ac1f6bac234a'
    'x-ms-version': 'REDACTED'
    'X-Content-Type-Options': 'REDACTED'
    'Date': 'Fri, 27 May 2022 03:19:28 GMT'
INFO:root:{'cat': 'STL', 'fn': 'Klay', 'sn': 'Thompson', 'pid': 202691, 'scr': 4, 'hs': 'https://ak-static.cms.nba.com/wp-content/uploads/headshots/nba/latest/260x190/202691.png'}
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://nbadatalakedev.table.core.windows.net/LeaderBoard(PartitionKey='0042100315',RowKey='1610612744_STL')'
Request method: 'PATCH'
Request headers:
    'x-ms-version': 'REDACTED'
    'DataServiceVersion': 'REDACTED'
    'Content

INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 204
Response headers:
    'Cache-Control': 'no-cache'
    'Content-Length': '0'
    'ETag': 'W/"datetime'2022-05-27T03%3A19%3A31.938847Z'"'
    'Server': 'Windows-Azure-Table/1.0 Microsoft-HTTPAPI/2.0'
    'x-ms-request-id': 'f7feefe9-7002-0046-2c78-71768a000000'
    'x-ms-client-request-id': 'd332463f-dd6b-11ec-a5a4-ac1f6bac234a'
    'x-ms-version': 'REDACTED'
    'X-Content-Type-Options': 'REDACTED'
    'Date': 'Fri, 27 May 2022 03:19:31 GMT'
INFO:root:GAMEON = True
INFO:root:LAST EVENT = 633
INFO:root:http://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2021/scores/pbp/0042100315_4_pbp.json
INFO:root:DESCRIPTION : stoppage: out-of-bounds
INFO:root:EVENT ID = 634
INFO:root:LENGTH OF SHOTS = 0
INFO:root:GAMEON = True
INFO:root:LAST EVENT = 634
INFO:root:http://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2021/scores/pbp/0042100315_4_pbp.json
INFO:root:WAITING FOR NEW DATA
INFO:root:WAITING FOR NEW DATA
INF

INFO:root:{'cat': 'REB', 'fn': 'Kevon', 'sn': 'Looney', 'pid': 1626172, 'scr': 17, 'hs': 'https://ak-static.cms.nba.com/wp-content/uploads/headshots/nba/latest/260x190/1626172.png'}
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://nbadatalakedev.table.core.windows.net/LeaderBoard(PartitionKey='0042100315',RowKey='1610612744_REB')'
Request method: 'PATCH'
Request headers:
    'x-ms-version': 'REDACTED'
    'DataServiceVersion': 'REDACTED'
    'Content-Type': 'application/json'
    'Accept': 'application/json'
    'Content-Length': '430'
    'x-ms-client-request-id': 'e19d62c4-dd6b-11ec-a2e0-ac1f6bac234a'
    'x-ms-date': 'REDACTED'
    'Date': 'Fri, 27 May 2022 03:19:55 GMT'
    'User-Agent': 'azsdk-python-data-tables/12.2.0 Python/3.9.0 (Linux-5.4.0-77-generic-x86_64-with-glibc2.31)'
    'Authorization': 'REDACTED'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 204
Response headers:
    'Cache-Control': 

INFO:root:{'cat': 'STL', 'fn': 'Sterling', 'sn': 'Brown', 'pid': 1628425, 'scr': 3, 'hs': 'https://ak-static.cms.nba.com/wp-content/uploads/headshots/nba/latest/260x190/1628425.png'}
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://nbadatalakedev.table.core.windows.net/LeaderBoard(PartitionKey='0042100315',RowKey='1610612742_STL')'
Request method: 'PATCH'
Request headers:
    'x-ms-version': 'REDACTED'
    'DataServiceVersion': 'REDACTED'
    'Content-Type': 'application/json'
    'Accept': 'application/json'
    'Content-Length': '431'
    'x-ms-client-request-id': 'e2fc845f-dd6b-11ec-9a0d-ac1f6bac234a'
    'x-ms-date': 'REDACTED'
    'Date': 'Fri, 27 May 2022 03:19:58 GMT'
    'User-Agent': 'azsdk-python-data-tables/12.2.0 Python/3.9.0 (Linux-5.4.0-77-generic-x86_64-with-glibc2.31)'
    'Authorization': 'REDACTED'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 204
Response headers:
    'Cache-Control':